In [134]:
import requests
import json  ### Intentar cambiar el formato JSON por uno más específico de Python
import sys
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

## Aceso a todas las variantes de un gen a través de la API de Ensembl

In [135]:
def acceso_variantes_ensembl (identificador_gen, especie):
    """ 
    DESCRIPCIÓN DE LA FUNCIÓN"""
    
    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/overlap/id/{identificador_gen}?feature=variation;species={especie}"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variantes = solicitud.json()
    
    return variantes



## Obtención de las variantes patogénicas a partir de las variantes que hemos obtenido de la API de Ensembl

In [136]:
def obtener_variantes_patogenicas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_patogenicas = []

    for var in variantes:
        if "clinical_significance" in var and "pathogenic" in var["clinical_significance"]:
            variantes_patogenicas.append(var)


    return variantes_patogenicas
            

## Obtención de las variantes benignas a partir de las variantes que hemos obtenido de la API de Ensembl

In [137]:
def obtener_variantes_benignas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_benignas = []

    for var in variantes:
        if "clinical_significance" in var and "benign" in var["clinical_significance"]:
            variantes_benignas.append(var)


    return variantes_benignas
            

In [138]:
variantes_benignas = obtener_variantes_benignas ("ENST00000269305", "human")

In [140]:
for i in variantes_benignas[:5]:
    print (i)

{'id': 'rs78378222', 'clinical_significance': ['uncertain significance', 'benign', 'risk factor'], 'assembly_name': 'GRCh38', 'start': 7668434, 'seq_region_name': '17', 'strand': 1, 'feature_type': 'variation', 'source': 'dbSNP', 'alleles': ['T', 'G'], 'consequence_type': '3_prime_UTR_variant', 'end': 7668434}
{'start': 7668539, 'clinical_significance': ['benign', 'likely benign'], 'assembly_name': 'GRCh38', 'id': 'rs114831472', 'end': 7668539, 'source': 'dbSNP', 'feature_type': 'variation', 'alleles': ['G', 'A', 'C'], 'consequence_type': '3_prime_UTR_variant', 'seq_region_name': '17', 'strand': 1}
{'seq_region_name': '17', 'strand': 1, 'alleles': ['C', 'A', 'G', 'T'], 'feature_type': 'variation', 'source': 'dbSNP', 'consequence_type': '3_prime_UTR_variant', 'end': 7668783, 'id': 'rs17884306', 'clinical_significance': ['benign'], 'assembly_name': 'GRCh38', 'start': 7668783}
{'seq_region_name': '17', 'strand': 1, 'consequence_type': '3_prime_UTR_variant', 'alleles': ['G', 'A', 'T'], 'fe

In [141]:
variantes_patogenicas = obtener_variantes_patogenicas ("ENST00000269305", "human")

In [142]:
for i in variantes_patogenicas[:5]:
    print (i)

{'clinical_significance': ['pathogenic'], 'seq_region_name': '17', 'id': 'rs730882017', 'feature_type': 'variation', 'end': 7669666, 'consequence_type': 'frameshift_variant', 'assembly_name': 'GRCh38', 'source': 'dbSNP', 'start': 7669666, 'alleles': ['C', '-'], 'strand': 1}
{'start': 7669673, 'alleles': ['TTTTT', 'TTTT'], 'strand': 1, 'feature_type': 'variation', 'end': 7669677, 'consequence_type': 'frameshift_variant', 'assembly_name': 'GRCh38', 'source': 'dbSNP', 'clinical_significance': ['uncertain significance', 'pathogenic'], 'seq_region_name': '17', 'id': 'rs2150988776'}
{'strand': 1, 'alleles': ['C', 'A', 'T'], 'start': 7669691, 'source': 'dbSNP', 'assembly_name': 'GRCh38', 'consequence_type': 'splice_acceptor_variant', 'feature_type': 'variation', 'end': 7669691, 'id': 'rs876658982', 'seq_region_name': '17', 'clinical_significance': ['likely pathogenic', 'pathogenic']}
{'feature_type': 'variation', 'end': 7669692, 'consequence_type': 'splice_acceptor_variant', 'assembly_name': 

## Obtención de la localización de las variantes

In [159]:
def obtener_localizacion_variantes (variantes):

    diccionario_posiciones_variantes = {}

    for var in variantes:
        inicio = var["start"]
        final = var["end"]
        longitud = final - (inicio - 1)
        diccionario_posiciones_variantes[var["id"]] = [inicio, final, longitud]
    
    return diccionario_posiciones_variantes
        

## Obtención de los exones

In [148]:
def obtener_exones_gen (identificador_transcrito, especie):
    
    rest_api = "https://rest.ensembl.org"
    extension_exones = f"/lookup/id/{identificador_transcrito}?expand=1&species={especie}"

    solicitud = requests.get(rest_api+extension_exones, headers={"Content-Type" : "application/json"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    exones = solicitud.json()

    return exones["Exon"]

    

In [149]:
exones = obtener_exones_gen ("ENST00000269305", "human")

for i in exones:
    print(i)

{'id': 'ENSE00003753508', 'strand': -1, 'seq_region_name': '17', 'species': 'homo_sapiens', 'version': 2, 'object_type': 'Exon', 'end': 7687490, 'start': 7687377, 'assembly_name': 'GRCh38', 'db_type': 'core'}
{'strand': -1, 'id': 'ENSE00004023728', 'object_type': 'Exon', 'seq_region_name': '17', 'species': 'homo_sapiens', 'version': 1, 'start': 7676521, 'db_type': 'core', 'assembly_name': 'GRCh38', 'end': 7676622}
{'strand': -1, 'id': 'ENSE00002419584', 'start': 7676382, 'assembly_name': 'GRCh38', 'db_type': 'core', 'end': 7676403, 'object_type': 'Exon', 'version': 1, 'seq_region_name': '17', 'species': 'homo_sapiens'}
{'id': 'ENSE00003625790', 'strand': -1, 'version': 1, 'seq_region_name': '17', 'species': 'homo_sapiens', 'object_type': 'Exon', 'end': 7676272, 'assembly_name': 'GRCh38', 'start': 7675994, 'db_type': 'core'}
{'id': 'ENSE00003518480', 'strand': -1, 'start': 7675053, 'assembly_name': 'GRCh38', 'db_type': 'core', 'end': 7675236, 'version': 1, 'seq_region_name': '17', 'spec

## Obtención de la secuencia de los exones en formato fasta

In [255]:
def obtener_secuencia_exones_fasta (identificador_transcrito, especie, nombre_archivo_fasta):
    exones = obtener_exones_gen (identificador_transcrito, especie)

    lista_secuencias = []

    contador = 1

    for ex in exones:
        identificador_exon = ex["id"]

        rest_api = "https://rest.ensembl.org"
        extension_secuencia_exon = f"/sequence/id/{identificador_exon}?content-type=text/x-fasta"

        solicitud2 = requests.get(rest_api+extension_secuencia_exon, headers={ "Content-Type" : "application/json"})

        if not solicitud2.ok:
            solicitud2.raise_for_status()
            sys.exit()

        exon = solicitud2.json()
        
        if ex["strand"] == -1:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]).complement())
            ex["strand"] = 1
        else:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]))
            
        secuencia_fasta.id = exon["id"]
        secuencia_fasta.name = exon["id"] + "exon" + str(contador)
        secuencia_fasta.description = exon["desc"]  + " exon " + str(contador)
        lista_secuencias.append(secuencia_fasta)

        contador +=1

    SeqIO.write(lista_secuencias, nombre_archivo_fasta, "fasta")

    print ("Se han guardado las secuencias correctamente")
    
    return exones


In [256]:
exones_fasta = obtener_secuencia_exones_fasta ("ENST00000269305", "human", "data/holu.fasta")


Se han guardado las secuencias correctamente


In [294]:
print (exones_fasta[1])

{'start': 7676521, 'id': 'ENSE00004023728', 'seq_region_name': '17', 'assembly_name': 'GRCh38', 'db_type': 'core', 'strand': 1, 'end': 7676622, 'version': 1, 'species': 'homo_sapiens', 'object_type': 'Exon'}


## Obtención de la localización de los exones

In [257]:
def obtener_localizacion_exones (exones):
    
    diccionario_posiciones_exones = {}

    for exon in exones["Exon"]:
        inicio = exon["start"]
        final = exon["end"]
        longitud = final - inicio
        diccionario_posiciones_exones[exon["id"]] = [inicio, final, longitud]
    
    return diccionario_posiciones_exones
        
    

## Obtención de las variantes que se encuentran dentro de los exones

In [312]:
secuencia = SeqIO.parse ("data/holu.fasta", "fasta")


In [313]:
secuencia1 = next(secuencia)



In [314]:
sec = "exon 1"

In [315]:
if secuencia1.description.find ("exon1") != -1:
    hola = 1
    print (hola)

In [316]:
print (secuencia1)
print (secuencia1.seq)

ID: ENSE00003753508
Name: ENSE00003753508
Description: ENSE00003753508 chromosome:GRCh38:17:7687377:7687490:-1 exon 1
Number of features: 0
Seq('GAGTTTTCAGATCTCGGTGGCAGGTCCCTCGTCCATCGACGACCCGAGGCCCCT...ACC')
GAGTTTTCAGATCTCGGTGGCAGGTCCCTCGTCCATCGACGACCCGAGGCCCCTGTGAAACGCAAGCCCGACCCTCGCACGAAAGGTGCTGCCACTGTGCGAAGGGACCTAACC


In [317]:
secuencia2 = next(secuencia)

In [318]:
print (secuencia2)
print (secuencia2.seq)

ID: ENSE00004023728
Name: ENSE00004023728
Description: ENSE00004023728 chromosome:GRCh38:17:7676521:7676622:-1 exon 2
Number of features: 0
Seq('GTCGGTCTGACGGAAGGCCCAGTGACGGTACCTCCTCGGCGTCAGTCTAGGATC...TGA')
GTCGGTCTGACGGAAGGCCCAGTGACGGTACCTCCTCGGCGTCAGTCTAGGATCGCAGCTCGGGGGAGACTCAGTCCTTTGTAAAAGTCTGGATACCTTTGA


In [265]:
print (7676537 - 7676521)
print (b[:15])
print (b[15:])

16
GTCGGTCTGACGGAA
GGCCCAGTGACGGTACCTCCTCGGCGTCAGTCTAGGATCGCAGCTCGGGGGAGACTCAGTCCTTTGTAAAAGTCTGGATACCTTTGA


## Obtener un diccionario con todas las variantes de un determinado exón 

In [319]:
def obtener_variantes_exon (variantes, identificador_exon, exones):
    for ex in exones:
        if ex["id"] == identificador_exon:
            inicio_exon = ex["start"]
            final_exon = ex["end"]

            diccionario_variantes_exon = {}

            for var in variantes:
                inicio_variante = var["start"]

                if inicio_exon <= inicio_variante <= final_exon:
                    diccionario_variantes_exon[var["id"]] = var

    return diccionario_variantes_exon
    

In [320]:
diccionario_variantes_exon = obtener_variantes_exon (variantes_patogenicas, "ENSE00004023728", exones)
print (diccionario_variantes_exon)

{'rs1131691018': {'strand': 1, 'start': 7676537, 'alleles': ['AAAA', 'AAA'], 'source': 'dbSNP', 'assembly_name': 'GRCh38', 'feature_type': 'variation', 'consequence_type': 'frameshift_variant', 'end': 7676540, 'id': 'rs1131691018', 'seq_region_name': '17', 'clinical_significance': ['pathogenic']}, 'rs876658627': {'id': 'rs876658627', 'seq_region_name': '17', 'clinical_significance': ['pathogenic'], 'source': 'dbSNP', 'assembly_name': 'GRCh38', 'consequence_type': 'frameshift_variant', 'feature_type': 'variation', 'end': 7676545, 'strand': 1, 'start': 7676543, 'alleles': ['TTT', 'TT']}, 'rs1555526997': {'start': 7676550, 'alleles': ['A', '-'], 'strand': 1, 'feature_type': 'variation', 'consequence_type': 'frameshift_variant', 'end': 7676550, 'assembly_name': 'GRCh38', 'source': 'dbSNP', 'clinical_significance': ['pathogenic'], 'seq_region_name': '17', 'id': 'rs1555526997'}, 'rs1555527002': {'consequence_type': 'frameshift_variant', 'feature_type': 'variation', 'end': 7676561, 'source': 

In [326]:
for i in diccionario_variantes_exon.values():
    print (len(i["alleles"][0]), len(i["alleles"][1]))

4 3
3 2
1 1
5 6


## Obtención de la secuencia de las variantes, tanto patogénicas, como benignas

Como al utilizar la API de Ensembl para obtener las variantes no obtenemos la secuencia completa del gen con estas variaciones, sino que solo obtenemos los alelos que han sufrido la variación lo que vamos a hacer es lo siguiente:

1. A partir del diccionario de las variantes que corresponden a un determinado exón, vamos a identificar el tipo de variación que ha sufrido la secuencia, una inserción (el alelo de referencia es más corto que el alternativo), una delección (el alelo de referencia tiene una mayor longitud que el alternativo) y un reemplazo (el alelo de referencia y el alternativo presentan la misma longitud). 
   
3. Realizamos la modificación de la secuencia del exón con el alelo mutado.  

In [385]:
def obtener_secuencia_variantes_exon (diccionario_variantes_exon, exones, exon_fasta):

    for ex in exones:
        if ex["id"] == exon_fasta.id:
            inicio_exon = ex["start"]
            final_exon = ex["end"]
            sec_exon_fasta = exon_fasta.seq
            
            secs =[]
            
            for var in diccionario_variantes_exon.values():
                if len(var["alleles"][0]) > len(var["alleles"][1]):
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"] - len(var["alleles"][0])
                    secuencia_variante = sec_exon_fasta[:inicio_variante+1]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante+1:]
                    secs.append (secuencia_variante)

                elif len(var["alleles"][0]) < len(var["alleles"][1]):
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"] - len(var["alleles"][0])
                    print (inicio_variante)
                    secuencia_variante = sec_exon_fasta[:inicio_variante+1]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante+1:]
                    secs.append (secuencia_variante)

                else:
                    longitud_variante = len(var["alleles"][0])
                    print (longitud_variante)
                    inicio_variante = final_exon - var["start"]
                    secuencia_variante = sec_exon_fasta[:inicio_variante]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante:]
                    secs.append (secuencia_variante)
                    
                    

    return secs
        

## Obtención de la secuencia de las variantes patogénicas en formato fasta

Como las variantes que hemos obtenido en un primer paso de obtención de variantes sabemos que corresponden a humanos, no necesitamso especificar la especie en este caso.

In [11]:
variant_id = "rs1333049"

# URL del punto final para obtener detalles de la variante
url = f"https://rest.ensembl.org/variation/human/{variant_id}?content-type=application/json"

# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    
    # Imprimir la estructura de la respuesta JSON para inspección
    print(data)

    # Verificar y procesar los datos para crear la secuencia FASTA
    if 'mappings' in data and len(data['mappings']) > 0:
        seq_region_name = data['mappings'][0]['seq_region_name']
        start = data['mappings'][0]['start']
        end = data['mappings'][0]['end']
        if 'alleles' in data:
            alleles = data['alleles']
            for allele in alleles:
                if 'allele' in allele:
                    allele_string = allele['allele']
                    # Crear la secuencia FASTA
                    fasta_header = f">{variant_id} {seq_region_name}:{start}-{end}"
                    fasta_sequence = f"{allele_string}"
                    fasta_output = f"{fasta_header}\n{fasta_sequence}"
                    
                    # Imprimir la secuencia en formato FASTA
                    print(fasta_output)
        else:
            print("No 'alleles' key found in the data.")
    else:
        print("No 'mappings' key found in the data or mappings list is empty.")
else:
    print(f"Error: {response.status_code}")

{'var_class': 'SNP', 'evidence': ['Frequency', '1000Genomes', 'Cited', 'Phenotype_or_Disease', 'TOPMed', 'gnomAD'], 'name': 'rs1333049', 'most_severe_consequence': 'intron_variant', 'ambiguity': 'V', 'minor_allele': None, 'source': 'Variants (including SNPs and indels) imported from dbSNP', 'clinical_significance': ['risk factor'], 'MAF': None, 'mappings': [{'coord_system': 'chromosome', 'seq_region_name': '9', 'end': 22125504, 'ancestral_allele': 'C', 'assembly_name': 'GRCh38', 'start': 22125504, 'allele_string': 'G/A/C', 'location': '9:22125504-22125504', 'strand': 1}], 'synonyms': ['PA166157726', 'VCV000812642', 'RCV001003460', 'rs58844516', 'rs59077428', 'rs17761501']}
No 'alleles' key found in the data.


In [13]:
variant_id = "rs1555523630"

# URL del punto final para obtener detalles de la variante
url = f"https://rest.ensembl.org/variation/human/{variant_id}?content-type=application/json"

# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    
    # Verificar y procesar los datos para crear la secuencia FASTA
    if 'mappings' in data and len(data['mappings']) > 0:
        mapping = data['mappings'][0]
        seq_region_name = mapping['seq_region_name']
        start = mapping['start']
        end = mapping['end']
        allele_string = mapping['allele_string']

        # Crear la secuencia FASTA para cada alelo
        alleles = allele_string.split('/')
        for allele in alleles:
            fasta_header = f">{variant_id} {seq_region_name}:{start}-{end}"
            fasta_sequence = allele
            fasta_output = f"{fasta_header}\n{fasta_sequence}"
            
            # Imprimir la secuencia en formato FASTA
            print(fasta_output)
else:
    print(f"Error: {response.status_code}")

>rs1555523630 17:7668203-7668204
CC
>rs1555523630 17:7668203-7668204
CCC


In [11]:
def get_sequence_with_variants(gene_id, variant_ids):
    server = "https://rest.ensembl.org"
    ext = f"/sequence/id/{gene_id}?type=genomic;var_synonyms={','.join(variant_ids)};content-type=text/x-fasta"
    
    response = requests.get(server + ext, headers={ "Content-Type" : "text/x-fasta"})
    
    if not response.ok:
        response.raise_for_status()
    
    return response.text

# Ejemplo de uso
gene_id = "ENSG00000012048"  # ID del gen BRCA1
variant_ids = ["rs80357713"]

sequence_with_variants = get_sequence_with_variants(gene_id, variant_ids)
print(sequence_with_variants)

>ENSG00000012048.26 chromosome:GRCh38:17:43044295:43170245:-1
...
AAACACTTCCA



In [77]:
def obtener_secuencia_variante (identificador_variante):

    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/variation/homo_sapiens/{identificador_variante}?content-type=application/json"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variante_informacion = solicitud.json()
    print (variante_informacion)
    
    #Ahora vamos a obtener la información relativa a la localización de la variante en la secuencia

    localizacion = variante_informacion["mappings"][0]
    cromosoma = localizacion["seq_region_name"]
    inicio_variante = localizacion["start"]
    fin_variante = localizacion["end"]

    flanco_secuencia = 50
    inicio_secuencia = inicio_variante - flanco_secuencia
    fin_secuencia = fin_variante + flanco_secuencia

    extension_secuencia_variantes = f"/sequence/region/human/{cromosoma}:{inicio_secuencia}..{fin_secuencia}:1?content-type=text/plain"
    solicitud2 = requests.get(rest_api+extension_secuencia_variantes, headers={"Content-Type": "text/plain"})

    if not solicitud2.ok:
        solicitud2.raise_for_status()
        sys.exit()

    secuencia_variante = solicitud2.text
        
    print (secuencia_variante)
    return secuencia_variante

        
    

In [79]:
obtener_secuencia_variante ("rs587781664")

{'evidence': ['Cited', 'Phenotype_or_Disease'], 'ambiguity': 'N', 'minor_allele': None, 'source': 'Variants (including SNPs and indels) imported from dbSNP', 'clinical_significance': ['not provided', 'likely pathogenic', 'pathogenic'], 'MAF': None, 'name': 'rs587781664', 'most_severe_consequence': 'splice_acceptor_variant', 'mappings': [{'allele_string': 'T/A/C/G', 'end': 7669692, 'strand': 1, 'assembly_name': 'GRCh38', 'location': '17:7669692-7669692', 'ancestral_allele': 'T', 'coord_system': 'chromosome', 'seq_region_name': '17', 'start': 7669692}], 'var_class': 'SNP', 'synonyms': ['VCV000635383', 'RCV000786810', 'VCV000141332', 'RCV000206779', 'RCV002288631', 'RCV000129814', 'VCV001264355', 'RCV000213070', 'RCV001667873', 'TP53_g.17908A>G']}
ACAAGAAGTGGAGAATGTCAGTCTGAGTCAGGCCCTTCTGTCTTGAACATGAGTTTTTTATGGCGGGAGGTAGACTGACCCTTTTTGGACTTCAGGTGGCTGTAGGAGACAGAAGCAGGGAGGAGAGATGACATCACATGAGTGAGAGGGTCTGTGCCCCTTTTCCCTGACCAATGCTTTGAAGGGCCTAAGGCTGGGACA


'ACAAGAAGTGGAGAATGTCAGTCTGAGTCAGGCCCTTCTGTCTTGAACATGAGTTTTTTATGGCGGGAGGTAGACTGACCCTTTTTGGACTTCAGGTGGCTGTAGGAGACAGAAGCAGGGAGGAGAGATGACATCACATGAGTGAGAGGGTCTGTGCCCCTTTTCCCTGACCAATGCTTTGAAGGGCCTAAGGCTGGGACA'

In [76]:

sec1 = "GGTGAGAGGCTGCCGAGGGGGAAGCAGCCCAGTCCTAGAAGCAGGGGAGGGAGAGAACCTTGTGCCAGCCGTGGCCGGAGGGGAGGAGGGACGGTTGGTTCCTGAGTTATGAATGGAGCCACCCCCTCCCTTGTTGTCATGCAGCCCGCAGACTGACCCAGTCTCCAGCCTTTGTTCCCCTCCCGGAACCCGGGCACTGGGC"
sec2 = "GGTGAGAGGCTGCCGAGGGGGAAGCAGCCCAGTCCTAGAAGCAGGGGAGGGAGAGAACCTTGTGCCAGCCGTGGCCGGAGGGGAGGAGGGACGGTTGGTTCCTGAGTTATGAATGGAGCCACCCCCTCCCTTGTTGTCATGCAGCCCGCAGACTGACCCAGTCTCCAGCCTTTGTTCCCCTCCCGGAACCCGGGCACTGGGC"
print (sec1 == sec2)

True


In [52]:
def variantes_region (identificador_gen, especie, cromosoma_gen, pos_inicio, pos_fin):
    variantes_patogenicas = obtener_variantes_patogenicas (identificador_gen, especie)
    diccionario_variantes = {}

    for i in variantes_patogenicas:
        rest_api = "https://rest.ensembl.org"
        extension_variantes = f"/variation/homo_sapiens/{identificador_variante}?content-type=application/json"

        solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


        if not solicitud.ok:
            solicitud.raise_for_status()
            sys.exit()

        variante_informacion = solicitud.json()
    
    #Ahora vamos a obtener la información relativa a la localización de la variante en la secuencia

        localizacion = variante_informacion["mappings"][0]
        cromosoma = localizacion["seq_region_name"]
        inicio_variante = localizacion["start"]
        fin_variante = localizacion["end"]


        if cromosoma == cromosoma_gen and inicio_variante > pos_inicio and inicio_variante < pos_fin and fin_variante > pos_inicio and fin_variante < pos_fin:
            diccionario_variantes[i["id"]] = i

    return diccionario_variantes


        

    
    

In [66]:
def guardar_fasta (identificador_variante):
    x = SeqRecord(Seq((obtener_secuencia_variante(identificador_variante))))
    SeqIO.write (x, "secuencia.fasta", "fasta")
    

In [67]:
def obtener_secuencia_gen(id_gen):
    rest_api = "https://rest.ensembl.org"
    extension_gen = f"/sequence/id/{id_gen}?content-type=text/plain"
    
    solicitud = requests.get(rest_api + extension_gen, headers={"Content-Type": "text/plain"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        return None

    secuencia_gen = solicitud.text

In [73]:
x = obtener_secuencia_gen ("ENSG00000141510")